In [1]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import scipy.optimize as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import math




from tabulate import tabulate
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, std

## Data Cleaning and Splitting

In [2]:
# load data
data=pd.read_csv('/Users/lsx/Desktop/5243project4/data/compas-scores-two-years.csv')



In [3]:
#since we are only intersted in sample fairness between two races: African-American and Caucasian
#and LFR needs to cageorize sensitive group and nonsensitive group,we regard defandant with African-American as 0,Caucasian as 1
data = data[(data['race']=='African-American') | (data['race']=='Caucasian')]
data = data.replace({'race': 'Caucasian'}, 1)
data = data.replace({'race': 'African-American'}, 0)

In [4]:
#drop variables with many NAs
data=data.drop(columns=['id', 'name', 'first', 'last','compas_screening_date','dob','age','c_jail_in', 'c_jail_out', 'c_case_number','c_offense_date','c_charge_desc', 'c_arrest_date','r_charge_desc',
'r_case_number','r_charge_desc','r_offense_date', 'r_jail_in', 'r_jail_out','violent_recid','vr_case_number',
'vr_offense_date', 'vr_charge_desc', 'screening_date','v_screening_date','in_custody','out_custody','r_charge_degree','r_days_from_arrest',
                      'vr_charge_degree','type_of_assessment','v_type_of_assessment' ])

In [5]:
data.shape

(6150, 22)

In [6]:
# drop NA 
data=data.dropna()

- Before building the LFR model, we need to transfer variables to dummy variables .
- Split data into Sensitive and Nonsensitive data

In [7]:
#Encode gender to 0/1
data['sex'].loc[data['sex']=='Male']= 1
data['sex'].loc[data['sex']=='Female']= 0

data['age_cat'].loc[data['age_cat']=='25 - 45']= 'B'
data['age_cat'].loc[data['age_cat']=='Greater than 45']= 'C'
data['age_cat'].loc[data['age_cat']=='Less than 25']= 'A'
data.loc[data['age_cat']=='A', 'age_cat1'] = 1
data.loc[data['age_cat']!='A', 'age_cat1'] = 0
data.loc[data['age_cat']=='B', 'age_cat2'] = 1
data.loc[data['age_cat']!='B', 'age_cat2'] = 0


data['c_charge_degree'].loc[data['c_charge_degree']=='M']= 1
data['c_charge_degree'].loc[data['c_charge_degree']=='F']= 0

data['v_score_text'].loc[data['v_score_text']=='High']= 'A'
data['v_score_text'].loc[data['v_score_text']=='Medium']= 'B'
data['v_score_text'].loc[data['v_score_text']=='Low']= 'C'
data.loc[data['v_score_text']=='A', 'v_score_text1'] = 1
data.loc[data['v_score_text']!='A', 'v_score_text1'] = 0
data.loc[data['v_score_text']=='B', 'v_score_text2'] = 1
data.loc[data['v_score_text']!='B', 'v_score_text2'] = 0


data['score_text'].loc[data['score_text']=='High']= 'A'
data['score_text'].loc[data['score_text']=='Low']= 'B'
data['score_text'].loc[data['score_text']=='Medium']= 'C'
data.loc[data['score_text']=='A', 'score_text1'] = 1
data.loc[data['score_text']!='A', 'score_text1'] = 0
data.loc[data['score_text']=='B', 'score_text2'] = 1
data.loc[data['score_text']!='B', 'score_text2'] = 0

/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3470841975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sex'].loc[data['sex']=='Female']= 0
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3470841975.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['age_cat'].loc[data['age_cat']=='Greater than 45']= 'C'
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3470841975.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [8]:
# drop original categorical variables
data=data.drop(columns=['age_cat','v_score_text','score_text'])

In [9]:
data.head()

,sex,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,c_charge_degree,...,start,end,event,two_year_recid,age_cat1,age_cat2,v_score_text1,v_score_text2,score_text1,score_text2
1,1,0,0,3,0,0,0,-1.0,1.0,0,...,9,159,1,1,0.0,1.0,0.0,0.0,0.0,1.0
2,1,0,0,4,0,1,4,-1.0,1.0,0,...,0,63,0,1,1.0,0.0,0.0,0.0,0.0,1.0
6,1,1,0,6,0,0,14,-1.0,1.0,0,...,5,40,1,1,0.0,1.0,0.0,0.0,0.0,0.0
8,0,1,0,1,0,0,0,-1.0,1.0,1,...,2,747,0,0,0.0,1.0,0.0,0.0,0.0,1.0
9,1,1,0,3,0,0,1,428.0,308.0,0,...,0,428,1,1,1.0,0.0,0.0,1.0,0.0,1.0


## Data Splitting

Split the dataset into training, validation and testing set (0.6, 0.2, 0.2)

In [10]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=1)
data_train, data_val= train_test_split(data_train, test_size=0.25, random_state=1)

In [11]:
data_train=pd.concat([data_train.sex,data_train.race,data_train.juv_fel_count,data_train.decile_score,data_train.two_year_recid],axis=1)
data_train=data_train.iloc[:20]
data_val=pd.concat([data_val.sex,data_val.race,data_val.juv_fel_count,data_val.decile_score,data_val.two_year_recid],axis=1)
data_test=pd.concat([data_test.sex,data_test.race,data_test.juv_fel_count,data_test.decile_score,data_test.two_year_recid],axis=1)

In [12]:
data_train.head()

,sex,race,juv_fel_count,decile_score,two_year_recid
246,1,1,0,1,0
3883,1,0,0,1,0
6410,1,0,0,2,0
437,1,1,0,10,1
3497,0,1,0,10,1


# Learning Fair Representations

$d(x_n, v_k, \alpha) = \sum^D_{d=1} \alpha_d (x_{nd} - v_{kd})^2$

In [13]:
# distance - d(x_n, v_k, alpha)
def distance(X, v, alpha):
    N = X.shape[0]
    D = X.shape[1]
    K = len(v)

    
    res = np.zeros((N, K))
    
    for n in range(N):
        for k in range(K):
            for d in range(D):
                res[n, k] += alpha[d]*(X.iloc[n][d] - v[k, d])**2
    
    return res

$M_{nk} = P(Z=k|x_n) \space\space \forall n, k \\ 
= exp(-d(x_n, v_k))/\sum_{k=1}^K exp(-d(x_n, v_k))$

In [14]:
# M_nk = P(Z=k|x_n)is the probablity that x_n maps to v_k

def M_nk(dist, k):
    
    N = dist.shape[0]
    K = dist.shape[1]
    M_nk = np.zeros((N, K))
    expo_res = np.zeros((N, K))
    
    for n in range(N):
        deno = 0
        for k in range(K):
            expo_res[n, k] = math.exp((-1)*dist[n, k])
            deno += expo_res[n, k]
        for k in range(K):
            M_nk[n, k] = expo_res[n, k] / deno
    return M_nk

$M_k = \frac{1}{|X_0|} \sum_{n \in X_0} M_{nk}$

$X_0 = \{X_0^+, X_0^-\}$

$X_0$ is training data

In [15]:
#  M_k
def M_k(X, M_nk, k):
    N = X.shape[0]
    K = M_nk.shape[1]
    M_k = np.zeros(K)
    
    for k in range(K):
        for n in range(N):
            M_k[k] += M_nk[n, k]
        M_k[k] = M_k[k]/N
    return M_k

$
\hat{x}_n = \sum^K_{k=1}M_{nk}v_k
$

$
L_x = \sum_{n=1}^N (x_n - \hat{x}_n)^2
$

In [16]:
#  the reconstruction of x_n  and L_x
def x_n_hat(X, M_nk, v):

    N = M_nk.shape[0]
    D = X.shape[1]
    K = M_nk.shape[1]
    x_n_hat = np.zeros((N, D))
    L_x = 0
    
    
    for n in range(N):
        for d in range(D):
            for k in range(K):
                x_n_hat[n, d] += M_nk[n, k]*v[k, d]
        # calculate L_x        
        L_x += (X.iloc[n][d] - x_n_hat[n, d])**2
    
    return x_n_hat, L_x

$
L_y = \sum_{n=1}^N -y_n log \hat{y}_n - (1-y_n)log(1- \hat{y}_n)
$


minimize $L = A_z L_Z + A_x L_x + A_y L_y$ 

$A_x, A_z, A_y$ are hyperparameters governing trade-off between the system desiderata

In [17]:
# prediction for y_n and L_y
def y_n_hat(M_nk, w, y):
    N = M_nk.shape[0]
    K = M_nk.shape[1]
    y_n_hat = np.zeros(N)
    L_y = 0
    

    for n in range(N):
        for k in range(K):
            y_n_hat[n] += M_nk[n, k]*w[k]
        # calculate L_y
        L_y += (-1)*y.iloc[n]*np.log(y_n_hat[n]) - (1 - y.iloc[n])*np.log(1 - y_n_hat[n])
        
    return y_n_hat, L_y

In [18]:
#  the metric function we wanna minimize
def L(param, sen_df, nsen_df, sen_y, nonsen_y, K, A_z, A_x, A_y):
    # param is the list of parameters
    # sen_df is the sensitive dataset
    # nsen_df is the nonsensitive dataset
    # sen_y is the list of labels for sensitive dataset
    # nonsen_y is the list of labels for nonsensitive dataset
    # K, A_z, A_x and A_y are hyperparameters, the values are decided by the users
    
    sen_N, sen_D = sen_df.shape
    nsen_N, nsen_D = nsen_df.shape

   
    alpha_sen = param[:sen_D]
    alpha_nsen = param[sen_D : 2 * sen_D]
    w = param[2 * sen_D : (2 * sen_D) + K]
    v = np.matrix(param[(2 * sen_D) + K:]).reshape((K, sen_D))

    #  the distance matrix
    dist_sen = distance(sen_df, v, alpha_sen)
    dist_nsen = distance(nsen_df, v, alpha_nsen)        

    # M_nk matrix
    M_nk_sen = M_nk(dist_sen, K)
    M_nk_nsen = M_nk(dist_nsen, K)

    #  M_k matrix
    M_k_sen = M_k(sen_df, M_nk_sen, K)
    M_k_nsen = M_k(nsen_df, M_nk_nsen, K)

    #  L_z
    L_z = 0
    for k in range(K):
        L_z += abs(M_k_sen[k] - M_k_nsen[k])

    #  x_n_hat and L_x
    x_n_hat_sen, L_x_sen = x_n_hat(sen_df, M_nk_sen, v)
    x_n_hat_nsen, L_x_nsen = x_n_hat(nsen_df, M_nk_nsen, v)
    L_x = L_x_sen + L_x_nsen

    # y_n_hat and L_y
    y_hat_sen, L_y_sen = y_n_hat(M_nk_sen, w, sen_y)
    y_hat_nsen, L_y_nsen = y_n_hat(M_nk_nsen, w, nonsen_y)
    L_y = L_y_sen + L_y_nsen

    # the  metric function
    metric = A_z*L_z + A_x*L_x + A_y*L_y

    return metric

$
\hat{y}_n = \sum^K_{k=1} M_{nk}w_k \\
0< w_k <1
$

In [19]:
# defines the threshold for y_n_hat to be 0 or 1
def predic_threshold(preds):
    for i in range(len(preds)):
        if preds[i] >= 0.5:
            preds[i] = 1
        else:
            preds[i] = 0
    return preds

In [20]:
#  calculate y_n_hat by using the best parameters
def cal_pred(params, D, K, sen_dt, nsen_dt, sen_label, nsen_label):
    # form parameters in new forms
    best_alpha_sen = params[:D]
    best_alpha_nsen = params[D : 2 * D]
    best_w = params[2 * D : (2 * D) + K]
    best_v = np.matrix(params[(2 * D) + K:]).reshape((K, D))
    
    # calculate the distance matrix
    best_dist_sen = distance(sen_dt, best_v, best_alpha_sen)
    best_dist_nsen = distance(nsen_dt, best_v, best_alpha_nsen) 
    
    # calculate the M_nk matrix
    best_M_nk_sen = M_nk(best_dist_sen, K)
    best_M_nk_nsen = M_nk(best_dist_nsen, K)
    
    # calculate the y_n_hat matrix
    y_hat_sen, L_y_sen = y_n_hat(best_M_nk_sen, best_w, sen_label)
    y_hat_nsen, L_y_nsen = y_n_hat(best_M_nk_nsen, best_w, nsen_label)
    
    return y_hat_sen, y_hat_nsen

In [21]:
#  calculates the  overall accuracy, 
# accuracy for  African-American group
# accuracy for  Caucasian group
# and calibration of the model
def cal_calibr(y_pred_sen, y_pred_nsen, y_sen_label, y_nsen_label):
    converted_y_hat_sen = predic_threshold(y_pred_sen)
    converted_y_hat_nsen = predic_threshold(y_pred_nsen)

    y_pred_sen = pd.DataFrame(converted_y_hat_sen)
    y_pred_nsen = pd.DataFrame(converted_y_hat_nsen)
     
    # calculate the accuracy
    acc_sen = accuracy_score(y_sen_label, y_pred_sen)
    acc_nsen = accuracy_score(y_nsen_label, y_pred_nsen)
    
    all_labels = y_sen_label.append(y_nsen_label)
    all_preds = y_pred_sen.append(y_pred_nsen)
    total_accuracy = accuracy_score(all_preds, all_labels)
    
    print("The overall accuracy  is: ", total_accuracy)
    print("The accuracy for African-American  is: ", acc_sen)
    print("The accuracy for Caucasian  is: ", acc_nsen)
    print("The calibration of the model is: ", abs(acc_sen-acc_nsen))

In [22]:
# LFR function 
#get train /validation accuracy
def LFR(training_data, val_data, y_name, sen_variable_name, K, A_z, A_x, A_y):
    # dividing the training dataset into sensitive and nonsensitive group
    sen_training = training_data[training_data[sen_variable_name]==0]
    nsen_training = training_data[training_data[sen_variable_name]==1]
    
    # dividing the validation dataset 
    sen_val = val_data[val_data[sen_variable_name]==0]
    nsen_val = val_data[val_data[sen_variable_name]==1]
    
    # delete sensitive variable 
    sen_training=sen_training.drop(columns=[sen_variable_name])
    sen_val=sen_val.drop(columns=[sen_variable_name])  
    nsen_training = nsen_training.drop(columns=[sen_variable_name])
    nsen_val = nsen_val.drop(columns=[sen_variable_name])
    
    # assign y labels 
    y_sen_training = sen_training[y_name]
    sen_training = sen_training.drop(columns=[y_name])
    y_sen_val = sen_val[y_name]
    sen_val = sen_val.drop(columns=[y_name])
    y_nsen_training = nsen_training[y_name]
    nsen_training = nsen_training.drop(columns=[y_name])
    y_nsen_val = nsen_val[y_name]
    nsen_val = nsen_val.drop(columns=[y_name])
    
    # pick random value
    # talpha and w  are between 0 and 1 and sum up to 1
    alpha_sen_1=np.random.random_sample((sen_training.shape[1],))
    alpha_nsen_1=np.random.random_sample((nsen_training.shape[1],))
    alpha_sen=alpha_sen_1/sum(alpha_sen_1)
    alpha_nsen=alpha_nsen_1/sum(alpha_nsen_1)
    w_1=np.random.random_sample((K,))
    w=w_1/sum(w_1)
    v=np.random.random((K, sen_training.shape[1]))
    
    
    initial = []
    initial.extend(alpha_sen)
    initial.extend(alpha_nsen)
    initial.extend(w)
    
    for item in v:
        initial.extend(item)
    initial = np.array(initial)

    # the boundary of the parameters
    bound=[]

    #  alpha and w are between 0 and 1 and sum up to 1
    for d in range(sen_training.shape[1]):
        bound.append((0, 1))

    for d in range(nsen_training.shape[1]):
        bound.append((0, 1))

    for k in range(K):
        bound.append((0, 1))

    
    for k in range(K):
        for d in range(sen_training.shape[1]):
            bound.append((None, None))
    
    # minimize the metric 
    para, min_L, d = optim.fmin_l_bfgs_b(L, x0=initial, epsilon=1e-5, 
                                         args=(sen_training, nsen_training, y_sen_training, 
                                               y_nsen_training, K, A_z, A_x, A_y), 
                                         bounds = bound, approx_grad=True, 
                                         maxfun=150000, maxiter=150000)
    
    # predict y_n_hat for the training dataset 
    y_hat_sen_tr, y_hat_nsen_tr = cal_pred(para, sen_training.shape[1], K, sen_training, 
             nsen_training, y_sen_training, y_nsen_training)

    print(" training accuracy:")
    cal_calibr(y_hat_sen_tr, y_hat_nsen_tr, y_sen_training, y_nsen_training)
    
    
    
    
    # predict y_n_hat for the validation dataset 
    y_hat_sen_val, y_hat_nsen_val = cal_pred(para, sen_val.shape[1], K, sen_val, 
             nsen_val, y_sen_val, y_nsen_val)  
    print("validation accuracy:")
    cal_calibr(y_hat_sen_val, y_hat_nsen_val, y_sen_val, y_nsen_val)    
    
    return  para

In [23]:

start = time.time()
para_test = LFR(data_train, data_val, 'two_year_recid', 'race', 10, 0.3, 0.3, 0.4)
end = time.time() 
print( f"The training time: {end-start}")

/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/2372853154.py:13: RuntimeWarning: divide by zero encountered in log
  L_y += (-1)*y.iloc[n]*np.log(y_n_hat[n]) - (1 - y.iloc[n])*np.log(1 - y_n_hat[n])
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/2372853154.py:13: RuntimeWarning: invalid value encountered in multiply
  L_y += (-1)*y.iloc[n]*np.log(y_n_hat[n]) - (1 - y.iloc[n])*np.log(1 - y_n_hat[n])


 training accuracy:


/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_labels = y_sen_label.append(y_nsen_label)
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_preds = y_pred_sen.append(y_pred_nsen)


The overall accuracy  is:  0.9
The accuracy for African-American  is:  0.8333333333333334
The accuracy for Caucasian  is:  1.0
The calibration of the model is:  0.16666666666666663
++++++++++++++++++++++++++++
validation accuracy:
For the validation set:
The overall accuracy  is:  0.6246830092983939
The accuracy for African-American  is:  0.6139860139860139
The accuracy for Caucasian  is:  0.6410256410256411
The calibration of the model is:  0.027039627039627145
The training time: 1093.6725301742554


/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_labels = y_sen_label.append(y_nsen_label)
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_preds = y_pred_sen.append(y_pred_nsen)


In [24]:
# Testing the LFR model
sen_test = data_test[data_test['race']==0]
nsen_test = data_test[data_test['race']==1]

sen_test=sen_test.drop(columns=['race'])
nsen_test = nsen_test.drop(columns=['race'])

y_sen_test = sen_test['two_year_recid']
sen_test = sen_test.drop(columns=['two_year_recid'])

y_nsen_test = nsen_test['two_year_recid']
nsen_test = nsen_test.drop(columns=['two_year_recid'])

start = time.time()
y_hat_sen_test, y_hat_nsen_test = cal_pred(para_test, sen_test.shape[1], 10, sen_test, 
             nsen_test, y_sen_test, y_nsen_test)
end = time.time() 
print( f"runtime of testing LFR model: {end-start}")
cal_calibr(y_hat_sen_test, y_hat_nsen_test, y_sen_test, y_nsen_test)

Testing time: 4.76779317855835
The overall accuracy  is:  0.665257819103973
The accuracy for African-American  is:  0.6494413407821229
The accuracy for Caucasian  is:  0.6895074946466809
The calibration of the model is:  0.04006615386455803


/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_labels = y_sen_label.append(y_nsen_label)
/var/folders/fk/z7v3jbsd74q31y7nyk6xvmj80000gn/T/ipykernel_25954/3778343658.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_preds = y_pred_sen.append(y_pred_nsen)


# Maximizing fairness under accuracy constriants

https://towardsdatascience.com/optimization-with-scipy-and-application-ideas-to-machine-learning-81d39c7938b8

## Conclusion